In [1]:
# The aim of this file is to extract yield values from the TECAN and map it to the correct file
import numpy as np
import csv
import copy

In [2]:
# Here I'm extracting TECAN into a dictionnary (for starters)

plate_name = "ORI"
result_name = "tecan_results"

# plate_name = "plate_AL_1b"
# result_name = "plate_AL_1b_raw"

CV = 30
folder_result = "raw_data"

localisation = "{}/{}.csv".format(folder_result, result_name)
data_source = "{}/{}_concentrations_reconstituted.csv".format(folder_result, plate_name)
export_place = "{}/{}_yield_and_std.csv".format(folder_result, plate_name)
all_together_place = "{}/{}_everything.csv".format(folder_result, plate_name)
draw_mean = "{}/{}_draw_mean.csv".format(folder_result, plate_name)
draw_std =  "{}/{}_draw_std.csv".format(folder_result, plate_name)
draw_ratio =  "{}/{}_draw_ratio.csv".format(folder_result, plate_name)
outliers =  "{}/{}_outliers.csv".format(folder_result, plate_name)
comments_file = "{}/{}_comments.txt".format(folder_result, plate_name)


In [3]:
my_current_data = np.genfromtxt(data_source, delimiter=',', skip_header  = 1, dtype = "float")

In [4]:
wells_information = {}

with open(data_source) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row["name"] == "":
            pass
        if row["name"] == "P3":
            print(row)
            wells_information[row["name"]] = row
        else:
            wells_information[row["name"]] = row
            if row["DNA"] == "0":
                print(row["name"])


A6
A12
A18
OrderedDict([('nad', '0.099'), ('folinic_acid', '0.034'), ('DNA', '50'), ('coa', '0.026'), ('RBS', '10'), ('peg', '2'), ('nucleo_mix', '0.75'), ('spermidin', '0.3'), ('pga', '15'), ('aa', '0.75'), ('trna', '0.02'), ('mg_gluta', '1.2'), ('hepes', '50'), ('camp', '0.375'), ('K_gluta', '20'), ('promoter', '10'), ('name', 'P3')])


In [5]:
def present_in_array(new_sample, array):
    present = False
    new_sample = np.reshape(np.array(new_sample), (1,16))
    for i in range(array.shape[0]):
        if np.array_equiv(array[i,:],new_sample):
            present = True
            break
    
            
    return(present, i)

# Extracting control and plate info separately information

In [6]:
control_array = None
plate_array = None
control_max_array = None

for well, row in wells_information.items():
    this_sample_conc = [row["nad"],
                        row["folinic_acid"], 
                        row["DNA"], 
                        row["coa"], 
                        row["RBS"], 
                        row["peg"], 
                        row["nucleo_mix"],
                        row["spermidin"],
                        row["pga"],
                        row["aa"],
                        row["trna"],
                        row["mg_gluta"],
                        row["hepes"],
                        row["camp"],
                        row["K_gluta"],
                        row["promoter"]
                         ]
    if well.startswith("A"):        
        if control_array is None:
            this_sample_conc.extend([well, '', ''])
            this_sample_conc = np.reshape(this_sample_conc, (1, 19))
            control_array = this_sample_conc 
        else:
            # already present in the array. 
            # Either file up a new line or...
            present, i = present_in_array(this_sample_conc, control_array[:, 0:16])
    #         present, i = present_in_array(this_sample_conc[:, 0:16], array_with_wells[:, 0:16])
            if not present:    
    #             if well == 'M21':
    #                 print(i)
                if well == 'A6':
                    print("Empty DNA row is {}".format(i))
                if float(row["DNA"]) == 0:
                    print("Empty DNA row (from row) is {}".format(i))
                if well == 'M6':
                    print("Reference row is {}".format(i))
                if float(row["K_gluta"]) == 40 and float(row["DNA"]) != 0:
                    print("Reference row (from row) is {}".format(i))
                this_sample_conc.extend([well, '', ''])
                this_sample_conc = np.reshape(this_sample_conc, (1, 19))
                control_array = np.concatenate((control_array, this_sample_conc), axis = 0)
            else:
                if control_array[i,17] == '':
                    control_array[i,17] = well
                elif control_array[i,18] == '':
                    control_array[i,18] = well
                else:
                    # print("{}, {}, {}".format(well, row, array_with_wells[i,:]))
                    print(row["name"])
                    print("this makes sense, it's the max")
    elif well.startswith("P"): 
        if control_max_array is None:
            this_sample_conc.extend([well, '', ''])
            this_sample_conc = np.reshape(this_sample_conc, (1, 19))
            control_max_array = this_sample_conc 
        else:
            # already present in the array. 
            # Either file up a new line or...
            present, i = present_in_array(this_sample_conc, control_max_array[:, 0:16])
    #         present, i = present_in_array(this_sample_conc[:, 0:16], array_with_wells[:, 0:16])
            if well == 'P3':
                print("P3 is {}".format(i))
            if not present: 
                this_sample_conc.extend([well, '', '']) 
                if well == 'P3':
                    print("P3 is {}".format(i))
                this_sample_conc = np.reshape(this_sample_conc, (1, 19))
                control_max_array = np.concatenate((control_max_array, this_sample_conc), axis = 0)
            else:
                if control_max_array[i,17] == '':
                    control_max_array[i,17] = well
                elif control_max_array[i,18] == '':
                    control_max_array[i,18] = well
                else:
                    # print("{}, {}, {}".format(well, row, array_with_wells[i,:]))
                    print(row["name"])
                    print("this makes sense, it's the max")
    else:
        if plate_array is None:
            this_sample_conc.extend([well, '', ''])
            this_sample_conc = np.reshape(this_sample_conc, (1, 19))
            plate_array = this_sample_conc
        else:
            # already present in the array. 
            # Either file up a new line or...
            present, i = present_in_array(this_sample_conc, plate_array[:, 0:16])
    #         present, i = present_in_array(this_sample_conc[:, 0:16], array_with_wells[:, 0:16])
            if not present:    
                if well == 'G3':
                    print("G3 is {}".format(i))
                if well == 'M6':
                    print("M6 is {}".format(i))
                if well == 'M18':
                    print("M18 is {}".format(i))
#                 if well == 'C8':
#                     print(i)
                this_sample_conc.extend([well, '', ''])
                this_sample_conc = np.reshape(this_sample_conc, (1, 19))
                plate_array = np.concatenate((plate_array, this_sample_conc), axis = 0)
            else:
                if plate_array[i,17] == '':
                    plate_array[i,17] = well
                elif plate_array[i,18] == '':
                    plate_array[i,18] = well
                else:
                    print("{} is already present and it a duplciate of well {}".format(row["name"], plate_array[i,16]))
                    this_sample_conc.extend([well, '', ''])
                    this_sample_conc = np.reshape(this_sample_conc, (1, 19))
                    plate_array = np.concatenate((this_sample_conc, plate_array), axis = 0)
                    # print(row)
                    # print("this makes sense, it's the max")
# print(plate_array.shape)
# print(plate_array[1:15, :])

Empty DNA row is 3
Empty DNA row (from row) is 3
Reference row (from row) is 4
P3 is 0
P3 is 0


In [7]:
print(control_array)
if control_array is None:
    whole_array = plate_array
else:
    print(control_array[4,])
    print(control_array[5,])
    try:
        whole_array = np.concatenate((control_array, plate_array, control_max_array), axis = 0)
    except ValueError:
        whole_array = np.concatenate((control_array, control_max_array), axis = 0)
    print(whole_array[4,])
    print(whole_array[5,])


[['0.033' '0.0068' '50' '0.026' '10' '2' '0.15' '0.1' '3' '0.15' '0.02'
  '0.4' '50' '0.075' '4' '10' 'A2' 'A8' 'A14']
 ['0.165' '0.0068' '50' '0.026' '10' '2' '0.15' '0.1' '9' '0.15' '0.02'
  '0.4' '50' '0.375' '12' '10' 'A3' 'A9' 'A15']
 ['0.099' '0.0204' '50' '0.078' '10' '2' '0.15' '0.5' '3' '0.45' '0.1'
  '1.2' '50' '0.225' '4' '10' 'A4' 'A10' 'A16']
 ['0.033' '0.0204' '50' '0.078' '10' '2' '0.15' '0.5' '9' '0.45' '0.06'
  '2' '50' '0.225' '12' '10' 'A5' 'A11' 'A17']
 ['0.33' '0.068' '0' '0.26' '10' '2' '1.5' '1' '30' '1.5' '0.2' '4' '50'
  '0.75' '40' '10' 'A6' 'A12' 'A18']
 ['0.33' '0.068' '50' '0.26' '10' '2' '1.5' '1' '30' '1.5' '0.2' '4' '50'
  '0.75' '40' '10' 'A7' 'A13' 'A19']]
['0.33' '0.068' '0' '0.26' '10' '2' '1.5' '1' '30' '1.5' '0.2' '4' '50'
 '0.75' '40' '10' 'A6' 'A12' 'A18']
['0.33' '0.068' '50' '0.26' '10' '2' '1.5' '1' '30' '1.5' '0.2' '4' '50'
 '0.75' '40' '10' 'A7' 'A13' 'A19']
['0.33' '0.068' '0' '0.26' '10' '2' '1.5' '1' '30' '1.5' '0.2' '4' '50'
 '0.75' '40'

In [8]:
print(whole_array.shape)

(13, 19)


# Replacing named wells with their values

In [9]:
name_yield_dictionnary = {}

with open(localisation) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        name_yield_dictionnary[row["name"]] = row["Time_5"]
        


In [10]:
print(name_yield_dictionnary)

{'A2': '17221', 'A3': '11499', 'A4': '14001', 'A5': '11031', 'A6': '5042', 'A7': '5662', 'A8': '17773', 'A9': '11781', 'A10': '14583', 'A11': '10973', 'A12': '4913', 'A13': '5817', 'A14': '17998', 'A15': '11664', 'A16': '13931', 'A17': '10634', 'A18': '4741', 'A19': '5905', 'A20': '359', 'A21': '353', 'A22': '355', 'A23': '375', 'A24': '60000', 'B2': '16127', 'B3': '12861', 'B4': '12519', 'B5': '9009', 'B6': '3884', 'B7': '5512', 'B8': '17251', 'B9': '14269', 'B10': '13137', 'B11': '9021', 'B12': '3944', 'B13': '5860', 'B14': '17541', 'B15': '13943', 'B16': '11672', 'B17': '9084', 'B18': '3763', 'B19': '6136', 'B20': '357', 'B21': '355', 'B22': '368', 'B23': '384', 'B24': '60000', 'C2': '10561', 'C3': '12136', 'C4': '10477', 'C5': '8298', 'C6': '4417', 'C7': '5236', 'C8': '12397', 'C9': '13424', 'C10': '10932', 'C11': '8225', 'C12': '4345', 'C13': '5111', 'C14': '12552', 'C15': '12232', 'C16': '10768', 'C17': '8223', 'C18': '4384', 'C19': '4898', 'C20': '363', 'C21': '365', 'C22': '365

In [11]:
wells_name_array = np.copy(whole_array[:,16:19])
# print(wells_name_array)

In [12]:
for i in range(whole_array.shape[0]):
    for j in range(16,19):
        try:
            whole_array[i,j] = name_yield_dictionnary[whole_array[i,j]]
        except KeyError:
            whole_array[i,j] = -10000


In [13]:
whole_array = whole_array.astype(np.float32)
print(whole_array.shape)
print(whole_array)

(13, 19)
[[3.3000e-02 6.8000e-03 5.0000e+01 2.6000e-02 1.0000e+01 2.0000e+00
  1.5000e-01 1.0000e-01 3.0000e+00 1.5000e-01 2.0000e-02 4.0000e-01
  5.0000e+01 7.5000e-02 4.0000e+00 1.0000e+01 1.7221e+04 1.7773e+04
  1.7998e+04]
 [1.6500e-01 6.8000e-03 5.0000e+01 2.6000e-02 1.0000e+01 2.0000e+00
  1.5000e-01 1.0000e-01 9.0000e+00 1.5000e-01 2.0000e-02 4.0000e-01
  5.0000e+01 3.7500e-01 1.2000e+01 1.0000e+01 1.1499e+04 1.1781e+04
  1.1664e+04]
 [9.9000e-02 2.0400e-02 5.0000e+01 7.8000e-02 1.0000e+01 2.0000e+00
  1.5000e-01 5.0000e-01 3.0000e+00 4.5000e-01 1.0000e-01 1.2000e+00
  5.0000e+01 2.2500e-01 4.0000e+00 1.0000e+01 1.4001e+04 1.4583e+04
  1.3931e+04]
 [3.3000e-02 2.0400e-02 5.0000e+01 7.8000e-02 1.0000e+01 2.0000e+00
  1.5000e-01 5.0000e-01 9.0000e+00 4.5000e-01 6.0000e-02 2.0000e+00
  5.0000e+01 2.2500e-01 1.2000e+01 1.0000e+01 1.1031e+04 1.0973e+04
  1.0634e+04]
 [3.3000e-01 6.8000e-02 0.0000e+00 2.6000e-01 1.0000e+01 2.0000e+00
  1.5000e+00 1.0000e+00 3.0000e+01 1.5000e+00 2.000

# Removing outliers

In [14]:
def remove_outlier_cv(row, cv):
    x = row[16:19]
    main_row = row[0:16]
    mean = np.mean(x)
    sd = np.std(x)
    ratio = sd/mean * 100
    if ratio > cv:
        min_index = np.argmin(x)
        max_index = np.argmax(x)
        other_point_arg = 3 - min_index - max_index
        if (x[other_point_arg] - x[min_index]) > (x[max_index] - x[other_point_arg]):
            # Distance between medium and low is above distance between max and medium: discard lowest
            new_x = np.concatenate((x[[other_point_arg, max_index]], np.array([-1])), axis = 0)
        else:
            new_x = np.concatenate((x[[min_index, other_point_arg]], np.array([-1])), axis = 0)
        return(True, np.concatenate((main_row, new_x), axis = 0), row)
    else:
        return(False, row, 0)

In [15]:
outliers_array = None
cleaned_array = None

for row in whole_array:
   # print(row.shape)
    outlier, row, outlier_row = remove_outlier_cv(row, cv = CV)
    if outlier:
        print(row[16:19])
        print(outlier_row[16:19])
        if outliers_array is None:
            outliers_array = np.reshape(outlier_row, (1, 19)) 
        else:
            outliers_array = np.concatenate((outliers_array, np.reshape(outlier_row, (1, 19))), axis = 0)
    if cleaned_array is None:
        cleaned_array = np.reshape(row, (1, 19)) 
    else:
        cleaned_array = np.concatenate((cleaned_array, np.reshape(row, (1, 19))), axis = 0)
    

In [16]:
print(outliers_array)
if outliers_array is None:
    number_outliers = 0
else:
    print(outliers_array.shape)
    number_outliers = outliers_array.shape[0]
number_total = whole_array.shape[0]
number_bis = cleaned_array.shape[0]
assert number_bis == number_total
percentage = number_outliers/number_total * 100
text = "There are {} outliers out of {} ({}%) for CV of {}".format(number_outliers, number_total, round(percentage, 2), CV)
with open(comments_file, "w") as file_handle:
    file_handle.write(text)
    

None


In [17]:
print(outliers_array)

None


In [18]:
outliers_as_dict = []
fieldnames = ["nad", "folinic_acid", "DNA", "coa", "RBS", "peg", "nucleo_mix", 
              "spermidin", "pga", "aa", "trna", "mg_gluta", "hepes", "camp", "K_gluta", "promoter", 
              "value_1", "value_2", "value_3", "plaque_name"]

if not outliers_array is None:
    for row in outliers_array:
        new_dict = {}
        new_dict["nad"] = round(float(row[0]), 5)
        new_dict["folinic_acid"] = round(float(row[1]), 5)
        new_dict["DNA"] = round(float(row[2]), 4)
        new_dict["coa"] = round(float(row[3]), 5)
        new_dict["RBS"] = round(float(row[4]), 4)
        new_dict["peg"] = round(float(row[5]), 5)
        new_dict["nucleo_mix"] = round(float(row[6]), 5)
        new_dict["spermidin"] = round(float(row[7]), 5)
        new_dict["pga"] = round(float(row[8]), 5)
        new_dict["aa"] = round(float(row[9]), 5)
        new_dict["trna"] = round(float(row[10]), 5)
        new_dict["mg_gluta"] = round(float(row[11]), 4)
        new_dict["hepes"] = round(float(row[12]), 4)
        new_dict["camp"] = round(float(row[13]), 4)
        new_dict["K_gluta"] = round(float(row[14]), 4)
        new_dict["promoter"] = round(float(row[15]), 4)
        new_dict["value_1"] = round(float(row[16]), 4)
        new_dict["value_2"] = round(float(row[17]), 4)
        new_dict["value_3"] = round(float(row[18]), 4)
        new_dict["plaque_name"] = plate_name
        outliers_as_dict.append(new_dict)

In [19]:
with open(outliers, "w") as csv_handle:
    csv_writer = csv.DictWriter(csv_handle, fieldnames, restval='', extrasaction='ignore')
    csv_writer.writeheader()
    for result in outliers_as_dict:
        csv_writer.writerow(result)
        # print(result)

# Calculating yield

In [20]:
print(cleaned_array.shape)

if control_array is None:
    print("controls are different")
    autofluo = whole_array[4,16:19]
    autofluo_reference = np.array([5042, 4913, 4741])
    max_extract = np.array([24606, 25235, 23426])
else:
    try: 
        max_extract = cleaned_array[109,16:19]  # Will be changed to reference extract in later versions of the code
    except:
        max_extract = cleaned_array[7,16:19]
        
    autofluo = whole_array[4,16:19]
    autofluo_reference = autofluo

print(autofluo)
print(max_extract)

(13, 19)
[5042. 4913. 4741.]
[24606. 25235. 23426.]


In [21]:
def calculate_yield_mean_sd(x, autofluo, ref_fluo, autofluo_reference = None):
    if autofluo_reference is None:
        autofluo_reference = autofluo
    combinations = []
    auto_value = np.mean(autofluo)
    auto_value_ref = np.mean(autofluo_reference)
    
    if ref_fluo[2] == -1:
        ref_fluo = ref_fluo[0:2] 
    ref_value = np.mean(ref_fluo)
    ref_value = np.mean(ref_fluo)
    if x[2] == -1:
        x = x[0:2]  
    for value_x in x:
        if value_x == np.array([-1]):
            pass
        else:
            normlised_value = (value_x - auto_value)/(ref_value - auto_value_ref)
            combinations.append(normlised_value)
    yield_mean = np.mean(combinations)
    yield_sd = np.std(combinations)
    return({"yield_mean": yield_mean, "yield_std": yield_sd})

In [22]:
autofluo_from_data = autofluo
max_from_data = max_extract

In [23]:
# Verifications 
print(calculate_yield_mean_sd(x = max_from_data, autofluo = autofluo_from_data, 
                              ref_fluo = max_from_data, autofluo_reference = autofluo_reference))
print(calculate_yield_mean_sd(x = autofluo_from_data, autofluo = autofluo_from_data, 
                              ref_fluo = max_from_data, autofluo_reference = autofluo_reference))

{'yield_mean': 1.0, 'yield_std': 0.03840742}
{'yield_mean': 8.381903e-09, 'yield_std': 0.0063154115}


In [24]:
new_array = cleaned_array[:,0:16]
full_array = cleaned_array[:,0:19]
mean_yield_list = []
std_yield_list = []
mean_list = []
std_list = []


for i in range(cleaned_array.shape[0]):
    if cleaned_array[i, 18] == -1:
        mean = np.mean(cleaned_array[i, 16:18])
        std = np.std(cleaned_array[i, 16:18])
    else:
        mean = np.mean(cleaned_array[i, 16:19])
        std = np.std(cleaned_array[i, 16:19])
    dict_results = calculate_yield_mean_sd(cleaned_array[i, 16:19], autofluo_from_data, max_from_data)
    yield_mean = dict_results["yield_mean"]
    yield_std = dict_results["yield_std"]
    mean_yield_list.append(yield_mean)
    std_yield_list.append(yield_std)
    mean_list.append(mean)
    std_list.append(std)
    

In [25]:
print(new_array.shape)
print(full_array.shape)
full_array = np.concatenate((full_array, wells_name_array), axis = 1)
# print(full_array)

(13, 16)
(13, 19)


In [26]:
mean_array = np.reshape(mean_list, (new_array.shape[0], 1))
std_array = np.reshape(std_list, (new_array.shape[0], 1))
mean_yield_array = np.reshape(mean_yield_list, (new_array.shape[0], 1))
std_yield_array = np.reshape(std_yield_list, (new_array.shape[0], 1))


In [27]:
array_for_saving = np.concatenate((new_array, mean_array, std_array, mean_yield_array, std_yield_array), axis = 1)
print(array_for_saving.shape)
array_for_saving_everything = np.concatenate((full_array, mean_array, std_array, mean_yield_array, std_yield_array), axis = 1)
print(array_for_saving_everything.shape)


(13, 20)
(13, 26)


In [28]:
# np.savetxt(export_place, array_for_saving, delimiter=";",fmt='%.5f')

# np.savetxt(all_together_place, array_for_saving_everything, delimiter=";",fmt='%.5f')

In [29]:
list_of_dict_everything = []
list_of_dict_ML = []
fieldnames = ["nad", "folinic_acid", "DNA", "coa", "RBS", "peg", "nucleo_mix", 
              "spermidin", "pga", "aa", "trna", "mg_gluta", "hepes", "camp", "K_gluta", "promoter", 
              "value_1", "value_2", "value_3", "well_1", "well_2", "well_3", "mean", "std", "yield", "yield_std",
             "plaque_name"]

for row in array_for_saving_everything:
    new_dict = {}
    new_dict["nad"] = round(float(row[0]), 5)
    new_dict["folinic_acid"] = round(float(row[1]), 5)
    new_dict["DNA"] = round(float(row[2]), 4)
    new_dict["coa"] = round(float(row[3]), 5)
    new_dict["RBS"] = round(float(row[4]), 4)
    new_dict["peg"] = round(float(row[5]), 5)
    new_dict["nucleo_mix"] = round(float(row[6]), 5)
    new_dict["spermidin"] = round(float(row[7]), 5)
    new_dict["pga"] = round(float(row[8]), 5)
    new_dict["aa"] = round(float(row[9]), 5)
    new_dict["trna"] = round(float(row[10]), 5)
    new_dict["mg_gluta"] = round(float(row[11]), 4)
    new_dict["hepes"] = round(float(row[12]), 4)
    new_dict["camp"] = round(float(row[13]), 4)
    new_dict["K_gluta"] = round(float(row[14]), 4)
    new_dict["promoter"] = round(float(row[15]), 4)
    new_dict["value_1"] = round(float(row[16]), 4)
    new_dict["value_2"] = round(float(row[17]), 4)
    new_dict["value_3"] = round(float(row[18]), 4)
    new_dict["well_1"] = row[19]
    new_dict["well_2"] = row[20]
    new_dict["well_3"] = row[21]
    new_dict["mean"] = round(float(row[22]), 1)
    new_dict["std"] = round(float(row[23]), 1)
    new_dict["yield"] = round(float(row[24]), 4)
    new_dict["yield_std"] = round(float(row[25]), 4)
    new_dict["plaque_name"] = plate_name
    if round(float(row[2]), 4) == 0:
        print("is zero")
        print(new_dict)
    else:
        list_of_dict_ML.append(new_dict)
    list_of_dict_everything.append(new_dict)


is zero
{'nad': 0.33, 'folinic_acid': 0.068, 'DNA': 0.0, 'coa': 0.26, 'RBS': 10.0, 'peg': 2.0, 'nucleo_mix': 1.5, 'spermidin': 1.0, 'pga': 30.0, 'aa': 1.5, 'trna': 0.2, 'mg_gluta': 4.0, 'hepes': 50.0, 'camp': 0.75, 'K_gluta': 40.0, 'promoter': 10.0, 'value_1': 5042.0, 'value_2': 4913.0, 'value_3': 4741.0, 'well_1': 'A6', 'well_2': 'A12', 'well_3': 'A18', 'mean': 4898.7, 'std': 123.3, 'yield': 0.0, 'yield_std': 0.0063, 'plaque_name': 'ORI'}


In [30]:
with open(all_together_place, "w") as csv_handle:
    csv_writer = csv.DictWriter(csv_handle, fieldnames, restval='', extrasaction='ignore')
    csv_writer.writeheader()
    for result in list_of_dict_everything:
        csv_writer.writerow(result)
        # print(result)

In [31]:
fieldnames_for_ml = ["nad", "folinic_acid", "coa", "nucleo_mix", 
                    "spermidin", "pga", "aa", "trna", "mg_gluta", "camp", "K_gluta", 
                     "yield", "yield_std"]
with open(export_place, "w") as csv_handle:
    csv_writer = csv.DictWriter(csv_handle, fieldnames_for_ml, restval='', extrasaction='ignore')
    csv_writer.writeheader()
    for result in list_of_dict_ML:
        csv_writer.writerow(result)
        # print(result)

In [32]:
# Draw the plaque - take from the instruction AA script for more ideas.
fieldnames_plate = ['row']
for i in range(2,24):
    fieldnames_plate.append(str(i))
print(fieldnames_plate)


['row', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']


In [33]:
mean_info, std_info, ratio_info = {}, {}, {}
# "well_1", "well_2", "well_3"
for element in list_of_dict_everything:
    mean_info[element["well_1"]] = element["mean"]
    std_info[element["well_1"]] = element["std"]
    ratio_info[element["well_1"]] = float(element["std"])/float(element["mean"]) * 100
    mean_info[element["well_2"]] = element["mean"]
    std_info[element["well_2"]] = element["std"]
    ratio_info[element["well_2"]] = float(element["std"])/float(element["mean"]) * 100
    mean_info[element["well_3"]] = element["mean"]
    std_info[element["well_3"]] = element["std"]
    ratio_info[element["well_3"]] = float(element["std"])/float(element["mean"]) * 100

In [34]:
with open(draw_mean, "w") as mean_drawing_file:
    writer = csv.DictWriter(mean_drawing_file, fieldnames=fieldnames_plate, restval='0')
    writer.writeheader()
    current_row = 'A'
    row = {}
    row["row"] = current_row
    for element in sorted(mean_info.keys()):
#         print(row)
#         print(element)
        if element.startswith(current_row):
            row[element[1:]] = mean_info[element]
        else:
            writer.writerow(row)
            current_row = element[0]
            row = {"row": current_row}
            row[element[1:]] = mean_info[element]
    writer.writerow(row)

    

In [35]:
with open(draw_std, "w") as std_drawing_file:
    writer = csv.DictWriter(std_drawing_file, fieldnames=fieldnames_plate, restval='0')
    writer.writeheader()
    current_row = 'A'
    row = {}
    row["row"] = current_row
    for element in sorted(std_info.keys()):
#         print(row)
#         print(element)
        if element.startswith(current_row):
            row[element[1:]] = std_info[element]
        else:
            writer.writerow(row)
            current_row = element[0]
            row = {"row": current_row}
            row[element[1:]] = std_info[element]
    writer.writerow(row)

In [36]:
with open(draw_ratio, "w") as ratio_drawing_file:
    writer = csv.DictWriter(ratio_drawing_file, fieldnames=fieldnames_plate, restval='0')
    writer.writeheader()
    current_row = 'A'
    row = {}
    row["row"] = current_row
    for element in sorted(ratio_info.keys()):
#         print(row)
#         print(element)
        if element.startswith(current_row):
            row[element[1:]] = ratio_info[element]
        else:
            writer.writerow(row)
            current_row = element[0]
            row = {"row": current_row}
            row[element[1:]] = ratio_info[element]
    writer.writerow(row)